In [21]:
import json
import datetime

In [22]:
%run lib.ipynb

In [23]:
with open( "./data/playoff-schedule.json") as f:
    schedule_data = json.load(f)

schedule = DictObj(schedule_data)

### Game Parser

In [71]:
def parseGameById( gameFileId ):

    with open( f"./output/games/game_{gameFileId}.json") as f:
        data = json.load(f)

    _gameObj = DictObj(data)
    
    home_reference = _gameObj.home.reference
    away_reference = _gameObj.away.reference
    
    report = {
        'periods': _gameObj.period,
        'teams':{
            'home': _gameObj.home.name,
            'away': _gameObj.away.name,
        },
        'score':{
            'home': _gameObj.home.points,
            'away': _gameObj.away.points,
        },
        'shots':{
            'home' : {},
            'away' : {},
        }
        
    }
    
    _shots_home = []
    _shots_away = []
    
    #step: periods - events
    for period in _gameObj.periods:
        for event in period.events:
            
            if( event.event_type != "shotmissed" and  
                event.event_type != "shotsaved" and 
                event.event_type != "penaltygoal" and
                event.event_type != "emptynetgoal" and 
               event.event_type != "penaltyshotmissed" and 
               event.event_type != "penaltyshotsaved" and 
                event.event_type != "owngoal" and 
                event.event_type != "goal" ):
                continue
                
            
            #step: time calculate
            secondsPeriodByType = seconds_in_period

            if( period.type == 'overtime'):
                 secondsPeriodByType = seconds_in_overtime
                        
            timeInPeriod = secondsPeriodByType - timeToSeconds(event.clock)
            timeInGame = 0
                
            if( period.sequence == 1):
                timeInGame = timeInPeriod
            elif ( period.sequence == 2 or period.sequence == 3):
                 timeInGame = ((period.number - 1) * secondsPeriodByType ) + timeInPeriod
            else:
                #step: overtime
                timeInGame = (seconds_in_period * 3 )  +  ((period.sequence - 4) * seconds_in_overtime ) + timeInPeriod
                
            #step: time calculate    
            _shot = {
                'saved': event.event_type == "shotsaved",
                'shot_type': '',
                'goal_zone': '',
                'time_in_game': timeInGame,
            }
            
            if(hasattr(event.details, 'shot_type')):
                _shot['shot_type'] = event.details.shot_type
            if(hasattr(event.details, 'goal_zone')):
                _shot['goal_zone'] = event.details.goal_zone
                    
            if(event.attribution.reference == home_reference):
                #step: home
                _shots_home.append(_shot)
            else:
                #step: away
                _shots_away.append(_shot)
            
        #step: join by type
        
        home_dict = {}
        away_dict = {}
        
        for homeShot in _shots_home:
            key = homeShot['shot_type']
            if( key in home_dict):
                home_dict[key] += 1
            else:
                home_dict[key] = 1
                
        for awayShot in _shots_away:
            key = awayShot['shot_type']
            if( key in away_dict):
                away_dict[key] += 1
            else:
                away_dict[key] = 1
   
        
    #step: add to report
   
    report['shots']['home'] = home_dict
    report['shots']['away'] = away_dict

    return report
    
    

In [72]:
parseGameById('4c9a706e-b53d-484c-95a3-a9cd7105bc63')

{'periods': 3,
 'teams': {'home': 'Hurricanes', 'away': 'Rangers'},
 'score': {'home': 3, 'away': 5},
 'shots': {'home': {'wrist': 33,
   'slap': 5,
   'snap': 15,
   'backhand': 7,
   'tip-in': 2,
   'wrap-around': 1},
  'away': {'wrist': 30,
   'slap': 6,
   'snap': 18,
   'backhand': 5,
   'tip-in': 6,
   'deflected': 1,
   'wrap-around': 1}}}

### Loop

In [73]:
def getRoundGroupData( groupItem, round ):
    
    teams = groupItem.teams
    team_a = teams[0]
    team_b = teams[1]
    
    _groupReport = {
        'team_a': team_a,
        'team_b': team_b,
        'round': round,
        'games_reports':[]
    }
        
    num_games = len(groupItem.games)
    
    for gameItem in groupItem.games:
        game_score = gameItem.score.split(':')
        
        score_a = game_score[0]
        score_b = game_score[1]

        game_id = gameItem.id
        
        reportData = parseGameById( game_id )
        
        dateNums = gameItem.date.split('.')
        x = datetime.datetime(2024, int(dateNums[1]), int(dateNums[0]))
        dateFormated = x.strftime("%Y-%m-%d")
        
        _reportObj = {
            'schedule_data': {
                'game': gameItem.game,
                'id': gameItem.id,
                'date': gameItem.date,
                'date_format': dateFormated
            },
            'report': reportData
        }
        
        _groupReport['games_reports'].append(_reportObj)
        
    file_name = f"./output/reports/widget-shots-types/widget-shots-types_{team_a}-{team_b}.json"
    with open(file_name, "w") as outfile:
        json.dump(_groupReport, outfile, indent=2)
    
    return _groupReport

# Init

In [76]:
# Change this and run:  0,1,2
roundIdx = 0 

roundItem = schedule.rounds[roundIdx]
round = roundItem.round

for groupItem in roundItem.groups:
    group_report = getRoundGroupData(groupItem, round)